In [84]:
from matplotlib import pyplot as plt
from matplotlib import dates as mDates
import pandas as pd
import numpy as np
import seaborn as sns
import os
import requests as r
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import chi2_contingency

### Hypotéza 1:
Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.
K ověření hypotézy využijte $\chi^2$ test s tím, že také určíte, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k fatální nehodě. V tom vám může pomoct “expected” výstup $\chi^2$
testu. Nerozlišujte počet následků, ale pouze nehody, kde došlo k jednomu a více úmrtí
(p13a  > 0).

In [85]:
df = pd.read_pickle("accidents.pkl.gz")[['p36', 'p13a']]
df = df[((df['p36'] == 0) | (df['p36'] == 1)) & (df['p13a'] >= 1)]
ct = pd.crosstab(df['p36'], df['p13a'])
display(ct)

p = chi2_contingency(ct)[1]
if p < 0.05:
    print("zamítame nulovou hypotézu: na dálnicích se umíralo se jinou pravděpodobností jako na 1. třídě")
else:
    print("příjimáme nulovou hypotézu: na dálnicích se umíralo se stejnou pravděpodobností jako na 1. třídě")
display(p)

p13a,1,2,3,4
p36,,,,
0,155,9,1,1
1,815,70,16,10


příjimáme nulovou hypotézu: na dálnicích se umíralo se stejnou pravděpodobností jako na 1. třídě


0.4335040105324415

### Hypotéza 2:
Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.
Využijte vhodný test a určete, zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost
na požadované hladině významnosti.

In [86]:
df = pd.read_pickle("accidents.pkl.gz")[['p45a', 'p53']]
df = df[((df['p45a'] == 2) | (df['p45a'] == 39))] # (df['p53'] > 0)
# p45a - znacka 2=audi 39=skoda
# p53 - skoda v stokorunach
ct = pd.crosstab(df['p45a'], df['p53'])
display(ct)

p = chi2_contingency(ct)[1]
if p < 0.05:
    print("zamítame nulovou hypotézu: na dálnicích se umíralo se jinou pravděpodobností jako na 1. třídě")
else:
    print("příjimáme nulovou hypotézu: na dálnicích se umíralo se stejnou pravděpodobností jako na 1. třídě")

p53,0,1,2,3,4,5,6,7,8,9,...,7000,7500,8000,8010,8500,9000,10000,12000,15000,20000
p45a,,,,,,,,,,,,,,,,,,,,,
2,764,12,1,1,0,31,0,0,0,0,...,9,1,4,0,0,4,10,2,3,4
39,8203,240,74,13,7,641,1,1,5,1,...,10,1,14,1,1,2,8,0,1,0


zamítame nulovou hypotézu: na dálnicích se umíralo se jinou pravděpodobností jako na 1. třídě
